# Modelo de demanda de energía eléctrica de Distribuidores

En este notebook, se realiza el experimento para definir y registrar el modelo. El tipo de modelo será un XGBRegressor y se realizará una búsqueda de hiperparámetros con Optuna y MLFlow. Una vez hallados los mejores hiperparámetros, se registrará el modelo en MLFlow para su puesta en producción.

## Configuración inicial

In [1]:
#-- Librerías utilizadas

import os
import io
import yaml
import json
import numpy as np
import pandas as pd
import datetime

from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

from xgboost import XGBRegressor

import optuna
from optuna.integration import MLflowCallback

import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

In [4]:
#-- Semilla para reproducibilidad
SEED = 42
np.random.seed(SEED)

#-- Variables necesarias para que funcione el registro en MLFlow
%env AWS_ACCESS_KEY_ID=minio   
%env AWS_SECRET_ACCESS_KEY=minio123 
%env MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
%env AWS_ENDPOINT_URL_S3=http://localhost:9000

env: AWS_ACCESS_KEY_ID=minio
env: AWS_SECRET_ACCESS_KEY=minio123
env: MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
env: AWS_ENDPOINT_URL_S3=http://localhost:9000


## Tratamiento básico de datos

In [50]:
#-- 1. Lectura de datos crudos desde repo de Github.
#-- 1.1. Se utilizan las variables de airflow para no hardcodear acá.
with open('./../airflow/config/variables.json', 'r', encoding='utf-8') as file:
    variables = json.load(file)

#-- 1.2. Datos de demanda
dem_df = dem_df = pd.read_csv(variables['dem_csv_url'])
dem_df['fecha'] = pd.to_datetime(dem_df['fecha'])
dem_df = dem_df.copy().loc[dem_df['anio_cal'] >= 2019]

#-- 1.3. Datos de temperatura
temp_df = pd.read_csv(variables['temp_csv_url'])
temp_df = temp_df.copy().loc[temp_df['fecha'] >= '2019-01-01']
temp_df['fecha'] = pd.to_datetime(temp_df['fecha'])

#-- 2. Join
df_full = pd.merge(dem_df, temp_df, left_on=['fecha', 'rge_nemo'], right_on=['fecha', 'region'])
df_full = df_full.rename(columns={'anio_cal':'year'})

cat_cols = ['mes', 'age_nemo', 'tipo_dia']
num_cols = ['year', 'tmed']
target = ['dem_dia']
df = df_full[cat_cols + num_cols + target].dropna()

#-- 3. Agregado mensual
df_mensual = df.groupby(['year'] + cat_cols).mean().round(2).reset_index()

num_cols = ['tmed']

#-- 4. Split train-test
X = df_mensual[cat_cols + num_cols]
y = df_mensual['dem_dia']  # Serie 1D
strata = df_mensual[cat_cols]

X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=SEED,
    stratify=strata
)

#-- 5. Split interno para validación (25% del train)
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train_raw, y_train, 
    test_size=0.25, 
    random_state=SEED,
    stratify=X_train_raw[cat_cols]
)

## Preprocesamiento

Se implementa definiendo una clase, para luego incorporarla en un pipeline. De esta forma, el preprocesamiento queda registrado en MLFlow junto con el modelo y se facilita la ejecución de producción, tomando tanto el modelo como el encoder de la misma fuente.

In [51]:
class DemandPreprocessor(BaseEstimator, TransformerMixin):
    """
    Preprocesador que implementa:
      - Codificación cíclica de 'mes' → mes_sin, mes_cos
      - Target mean encoding de 'age_nemo' + 'tipo_dia' → dist_tipodia_te, con fallback a media global
      - Crea columna 'tmed2' (cuadrado de la temperatura).
      - Elimina columnas originales: 'age_nemo', 'tipo_dia', 'dist_tipodia' y 'mes'.
    """
    def __init__(self, period=12):
        self.period = period
        self.mapping_ = None
        self.global_mean_ = None

    def fit(self, X, y=None):
        X = X.copy()
        # Construcción de dist_tipodia para mapping
        X['dist_tipodia'] = X['age_nemo'].astype(str) + '_' + X['tipo_dia'].astype(str)

        if y is None:
            raise ValueError("Target y es requerido para target mean encoding.")
        df_te = pd.concat([X[['dist_tipodia']], y.rename('dem_dia')], axis=1)
        self.mapping_ = df_te.groupby('dist_tipodia')['dem_dia'].mean()  # Serie
        self.global_mean_ = float(y.mean())
        return self

    def transform(self, X):
        X = X.copy()

        # Codificación cíclica de mes
        X['mes_sin'] = np.sin(2 * np.pi * X['mes'] / self.period)
        X['mes_cos'] = np.cos(2 * np.pi * X['mes'] / self.period)

        # Target encoding
        X['dist_tipodia'] = X['age_nemo'].astype(str) + '_' + X['tipo_dia'].astype(str)
        X['dist_tipodia_te'] = X['dist_tipodia'].map(self.mapping_)
        X['dist_tipodia_te'] = X['dist_tipodia_te'].fillna(self.global_mean_)

        # Temperatura al cuadrado
        X['tmed2'] = X['tmed'] ** 2

        # Eliminamos columnas originales categóricas y la combinada
        X = X.drop(columns=['age_nemo', 'tipo_dia', 'dist_tipodia', 'mes'])

        # Orden de features
        ordered_cols = ['mes_sin', 'mes_cos', 'tmed', 'tmed2', 'dist_tipodia_te']
        return X[ordered_cols]

## Definición del pipeline (preprocesador + modelo)

In [52]:
def build_pipeline(trial=None):
    """Construye el pipeline con XGBRegressor y, opcionalmente, hiperparámetros propuestos por Optuna."""
    # Hiperparámetros buscados por Optuna
    if trial is not None:
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 500, 1200),
            "max_depth": trial.suggest_int("max_depth", 5, 10),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1, log=True),
            "subsample": trial.suggest_float("subsample", 0.7, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.8, 0.9),
            "min_child_weight": trial.suggest_float("min_child_weight", 1.0, 8.0),
            "gamma": trial.suggest_float("gamma", 0.0, 1.0),
            "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
            "reg_lambda": trial.suggest_float("reg_lambda", 1.0, 1.5),
        }
    else:
        # Valores por defecto razonables
        params = {
            "n_estimators": 500,
            "max_depth": 6,
            "learning_rate": 0.05,
            "subsample": 0.8,
            "colsample_bytree": 0.8,
            "min_child_weight": 1.0,
            "gamma": 0.0,
            "reg_alpha": 0.0,
            "reg_lambda": 1.0,
        }

    model = XGBRegressor(
        booster='gbtree',
        objective='reg:squarederror',
        eval_metric='mae',
        random_state=SEED,
        **params
    )

    pipeline = Pipeline(steps=[
        ("pre", DemandPreprocessor(period=12)),
        ("model", model)
    ])
    
    return pipeline, params

## Búsqueda de hiperparámetros

In [53]:
#-- 0. Configuración de MLFlow
mlflow.set_tracking_uri("http://localhost:5001")
experiment_name = 'Demanda Distribuidores'

def get_or_create_experiment(experiment_name):
    """
    Retrieve the ID of an existing MLflow experiment or create a new one if it doesn't exist.

    This function checks if an experiment with the given name exists within MLflow.
    If it does, the function returns its ID. If not, it creates a new experiment
    with the provided name and returns its ID.

    Parameters:
    - experiment_name (str): Name of the MLflow experiment.

    Returns:
    - str: ID of the existing or newly created MLflow experiment.
    """

    if experiment := mlflow.get_experiment_by_name(experiment_name):
        return experiment.experiment_id
    else:
        return mlflow.create_experiment(experiment_name)

experiment_id = get_or_create_experiment(experiment_name)
mlflow.set_experiment(experiment_id=experiment_id)

#-- 1. Definición de función para cada trial de optuna
def objective(trial):
    """Función objetivo de Optuna. Cada trial se loguea como run Hijo en MLflow."""
    pipeline, params = build_pipeline(trial)

    # Crear run hijo anidado para este trial
    with mlflow.start_run(run_name=f"trial_{trial.number}", nested=True):
        # Log de parámetros propuestos
        mlflow.log_params(params)

        # Entrenar y evaluar en validación
        pipeline.fit(X_tr, y_tr)

        y_pred_val = pipeline.predict(X_val)
        mae_val = mean_absolute_error(y_val, y_pred_val)
        mape_val = mean_absolute_percentage_error(y_val, y_pred_val)

        # Log de métricas del trial
        mlflow.log_metric("mae_val", float(mae_val))
        mlflow.log_metric("mape_val", float(mape_val))

        # Guardar como atributos del trial (para Optuna)
        trial.set_user_attr("mae_val", float(mae_val))
        trial.set_user_attr("mape_val", float(mape_val))

        # Retornar métrica objetivo (tu elección actual: MAE)
        return mae_val
        
#-- 2. Run padre para búsqueda de hiperparámetros
with mlflow.start_run(run_name="hyperparam_search_" + datetime.datetime.today().strftime('%Y-%m-%d_%H:%M:%S"'), description="Búsqueda de hiperparámetros con Optuna"):
    # Tags útiles en el run padre
    mlflow.set_tags({
        "owner": "MLOPS1",
        "stage": "hyperparam_search",
        "framework": "xgboost+sklearn",
        "optimizer": "optuna",
        "metric_objective": "mae",
    })

    # Creación del estudio Optuna
    study = optuna.create_study(
        direction="minimize",
        study_name=experiment_name
    )
    study.sampler = optuna.samplers.TPESampler(seed=SEED)
    study.pruner = optuna.pruners.MedianPruner(n_warmup_steps=10)

    # Ejecutar con runs hijos anidados
    study.optimize(objective, n_trials=100)

    best_trial = study.best_trial

    # Loguear resumen del estudio en el run padre
    mlflow.log_metric("best_value_mae_val", float(best_trial.value))
    mlflow.log_dict(
        {
            "best_trial_number": best_trial.number,
            "best_params": best_trial.params,
            "best_user_attrs": best_trial.user_attrs,
            "n_trials": len(study.trials),
        },
        artifact_file="study_summary.json"
    )

    print(f"Mejor MAE (valid): {best_trial.value:.4f}")
    print("Mejores params:", best_trial.params)

#-- 3. Run Padre: Best Model
# Entrenamiento final con los mejores hiperparámetros y log completo
best_params = best_trial.params

best_pipeline, _ = build_pipeline(trial=None)
best_pipeline.set_params(**{f"model__{k}": v for k, v in best_params.items()})

with mlflow.start_run(run_name="best_model", description="Entrenamiento final y evaluación en test"):
    mlflow.set_tags({
        "owner": "MLOPS1",
        "stage": "training_final",
        "framework": "xgboost+sklearn",
        "encoding": "cyclic_month + target_mean(age_nemo,tipo_dia)",
    })
    mlflow.log_params(best_params)

    # Fit final con todo el train “externo”
    best_pipeline.fit(X_train_raw, y_train)

    # Evaluación en test (reservado)
    y_pred_test = best_pipeline.predict(X_test_raw)
    mae_test = mean_absolute_error(y_test, y_pred_test)
    mape_test = mean_absolute_percentage_error(y_test, y_pred_test)

    mlflow.log_metric("mae_test", float(mae_test))
    mlflow.log_metric("mape_test", float(mape_test))

    # Firma e input_example (útil para serving)
    input_example = pd.DataFrame(X_train_raw).head(5)
    signature = infer_signature(input_example, best_pipeline.predict(input_example))

    # Log del modelo como artefacto 'best_model'
    info = mlflow.sklearn.log_model(
        sk_model=best_pipeline,
        artifact_path="best_model",
        signature=signature,
        input_example=input_example
    )

    # URI del artefacto
    model_uri = info.model_uri  # p.ej. 'runs:/<run_id>/best_model'
    run_id = info.run_id

    #model_uri = mlflow.get_artifact_uri("best_model")

    print(f"Best model logueado. MAE test: {mae_test:.4f} | MAPE test: {mape_test:.4f}")


[I 2025-12-04 16:14:08,183] A new study created in memory with name: Demanda Distribuidores
[I 2025-12-04 16:14:11,496] Trial 0 finished with value: 244.50440480510875 and parameters: {'n_estimators': 762, 'max_depth': 10, 'learning_rate': 0.05395030966670229, 'subsample': 0.8795975452591109, 'colsample_bytree': 0.8156018640442437, 'min_child_weight': 2.0919616423534184, 'gamma': 0.05808361216819946, 'reg_alpha': 0.8661761457749352, 'reg_lambda': 1.3005575058716043}. Best is trial 0 with value: 244.50440480510875.


🏃 View run trial_0 at: http://localhost:5001/#/experiments/5/runs/da3a6576221a456abdd5440b85d614ae
🧪 View experiment at: http://localhost:5001/#/experiments/5


[I 2025-12-04 16:14:13,351] Trial 1 finished with value: 208.32987233791658 and parameters: {'n_estimators': 996, 'max_depth': 5, 'learning_rate': 0.09330606024425668, 'subsample': 0.9497327922401265, 'colsample_bytree': 0.8212339110678276, 'min_child_weight': 2.272774770449704, 'gamma': 0.18340450985343382, 'reg_alpha': 0.3042422429595377, 'reg_lambda': 1.262378215816119}. Best is trial 1 with value: 208.32987233791658.


🏃 View run trial_1 at: http://localhost:5001/#/experiments/5/runs/0bf3d80f24e54400bc21e041447bc4a5
🧪 View experiment at: http://localhost:5001/#/experiments/5


[I 2025-12-04 16:14:15,019] Trial 2 finished with value: 226.63247450967407 and parameters: {'n_estimators': 802, 'max_depth': 6, 'learning_rate': 0.04091220574443785, 'subsample': 0.7418481581956126, 'colsample_bytree': 0.8292144648535219, 'min_child_weight': 3.564532903055842, 'gamma': 0.45606998421703593, 'reg_alpha': 0.7851759613930136, 'reg_lambda': 1.0998368910791798}. Best is trial 1 with value: 208.32987233791658.


...

🏃 View run trial_98 at: http://localhost:5001/#/experiments/5/runs/9dae939c59c14f92a1da953e076a5fb8
🧪 View experiment at: http://localhost:5001/#/experiments/5


[I 2025-12-04 16:18:58,231] Trial 99 finished with value: 210.12369472813737 and parameters: {'n_estimators': 1037, 'max_depth': 5, 'learning_rate': 0.08164470162933438, 'subsample': 0.9633762464482968, 'colsample_bytree': 0.8705425998259448, 'min_child_weight': 1.4105480511150827, 'gamma': 0.8182566636117428, 'reg_alpha': 0.8569265259603319, 'reg_lambda': 1.2122286888367408}. Best is trial 95 with value: 207.21099732639746.


🏃 View run trial_99 at: http://localhost:5001/#/experiments/5/runs/5994c64e6bc24117a156be848c364117
🧪 View experiment at: http://localhost:5001/#/experiments/5
Mejor MAE (valid): 207.2110
Mejores params: {'n_estimators': 1073, 'max_depth': 5, 'learning_rate': 0.09433373358733983, 'subsample': 0.9624756557534352, 'colsample_bytree': 0.8641384722517509, 'min_child_weight': 1.0655977064806268, 'gamma': 0.898734595076509, 'reg_alpha': 0.8314558143446663, 'reg_lambda': 1.1821091349126074}
🏃 View run hyperparam_search_2025-12-04_16:14:08" at: http://localhost:5001/#/experiments/5/runs/011ebf5ba7f740928708d973d64bf935
🧪 View experiment at: http://localhost:5001/#/experiments/5


/mnt/DATOS_RAID1/Repositorios/amq2-service-ml/.venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/12/04 16:19:00 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/04 16:19:11 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip

Best model logueado. MAE test: 196.0295 | MAPE test: 0.1370
🏃 View run best_model at: http://localhost:5001/#/experiments/5/runs/0174ef05a0f44fb6b72e811acc7ef8a0
🧪 View experiment at: http://localhost:5001/#/experiments/5


## Registro del modelo para producción

In [54]:
from mlflow import MlflowClient

client = MlflowClient()

registered_name = "demanda_distribuidores"
desc = "XGB + meses con encoding cíclico + target mean para distribuidor y tipo de día"
alias = "champion"

# Crear el modelo en el registry si no existe
try:
    client.create_registered_model(name=registered_name, description=desc)
except mlflow.exceptions.RestException:
    pass

# Tags útiles
tags = {**best_params, "mae_test": mae_test, "mape_test": mape_test, "model": type(best_pipeline.named_steps['model']).__name__}

# Crear versión y asignar alias
result = client.create_model_version(
    name=registered_name,
    source=model_uri,
    run_id=run_id,
    tags=tags
)
client.set_registered_model_alias(registered_name, alias, result.version)
print(f"Registrado '{registered_name}' v{result.version} como '{alias}'.")


2025/12/04 16:20:04 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: demanda_distribuidores, version 2


Registrado 'demanda_distribuidores' v2 como 'champion'.


## Verificación de predicciones

In [ ]:
#-- Carga del pipeline usando el nombre del modelo registrado y su alias

#-- Cliente de MLFlow
client_mlflow = mlflow.MlflowClient()

#-- Carga de los datos del modelo (es la estructura general registrada en MLFlow)
model_data_mlflow = client_mlflow.get_model_version_by_alias(registered_name, alias) #demanda_distribuidores / champion

#-- Carga del modelo propiamente dicho, en este caso es el pipeline que incluye el preprocesador y el modelo
pipeline = mlflow.sklearn.load_model(model_data_mlflow.source)


In [ ]:
#-- Ver signature del pipeline, para identificar las entradas
#-- esta línea no instancia el modelo para hacer predicciones, si no que carga los metadatos registrados.
mlflow_model = mlflow.models.Model.load(model_data_mlflow.source)
print("Signature:", mlflow_model.signature)
print("Inputs:", mlflow_model.signature.inputs)


Signature: inputs: 
  ['mes': long (required), 'age_nemo': string (required), 'tipo_dia': long (required), 'tmed': double (required)]
outputs: 
  [Tensor('float32', (-1,))]
params: 
  None

Inputs: ['mes': long (required), 'age_nemo': string (required), 'tipo_dia': long (required), 'tmed': double (required)]


In [ ]:
#-- Inspección del mapeo del target encoding guardado en el preprocesador, útil para buscar nombres de distribuidores
pre = pipeline.named_steps['pre']
age_map = pre.mapping_.reset_index()
age_map['dist_tipodia'].loc[age_map['dist_tipodia'].str.startswith('EPE')]

201    EPECORXD_1
202    EPECORXD_2
203    EPECORXD_3
204    EPENEUQD_1
205    EPENEUQD_2
206    EPENEUQD_3
207    EPESAFSD_1
208    EPESAFSD_2
209    EPESAFSD_3
Name: dist_tipodia, dtype: object

In [ ]:
#-- Ejecución de predicciones
df_infer = pd.DataFrame([
    {"mes": 9,  "age_nemo": "EPESAFSD", "tipo_dia": 3, "tmed": 15.5},
    {"mes": 1,  "age_nemo": "EDENOROD", "tipo_dia": 1, "tmed": 27.0},
    {"mes": 12, "age_nemo": "EDESURCD", "tipo_dia": 1, "tmed": 25.0},
])


pipeline.predict(df_infer)

array([21151.457, 79626.8  , 59258.293], dtype=float32)